In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt

# Szablon adresu URL (podmieniamy numer strony)
base_url = "https://wydawnictwopolitechniki.pl/;s,wyniki,dp,22,kat,22,tryb,normal,zacz,{}"

ksiazki = []
page = 1

while True:
    url = base_url.format(page)
    print(f"Pobieram stronę: {url}")
    r = requests.get(url)
    if r.status_code != 200:
        print("Błąd połączenia, kończę.")
        break

    soup = BeautifulSoup(r.text, "html.parser")

    # znajdź wszystkie książki na stronie
    tytuly = soup.find_all("h1", class_="tytul + tytul_normal")
    autorzy = soup.find_all("div", class_="autor + autor_normal")
    ceny = soup.find_all("div", class_="mybook_cenabox + mybook_cenabox_normal")

    # jeśli nie ma żadnych książek -> koniec paginacji
    if not tytuly:
        print("Brak książek, koniec.")
        break

    # iteruj po elementach
    for t, a, c in zip(tytuly, autorzy, ceny):
        title = t.get_text(strip=True)
        author_list = [x.strip() for x in a.get_text(strip=True).replace("Autor:", "").split(",")]
        price_text = c.get_text(strip=True).replace("zł", "").replace(",", ".")
        try:
            price = float(price_text)
        except ValueError:
            price = None

        ksiazki.append({
            "title": title,
            "author": author_list,
            "price": price
        })

    page += 1

print(f"\nZnaleziono {len(ksiazki)} książek.")